# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:

data_path="output_data/cities.csv"
cities=pd.read_csv(data_path)
cities

,Unnamed: 0,City,Country,Latitude,Longitude,Date Time,Temperature,Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,Bluff,NZ,-46.60,168.33,1595011950,35.73,88,25,6.02
1,1,La Ronge,CA,55.10,-105.28,1595011950,59.00,93,90,9.17
2,2,Port Alfred,ZA,-33.59,26.89,1595011951,64.76,33,0,9.78
3,3,Pochutla,MX,15.74,-96.47,1595011951,91.40,49,20,5.66
4,4,Whitianga,NZ,-36.83,175.70,1595011951,55.53,85,99,9.60
...,...,...,...,...,...,...,...,...,...,...
541,541,Bauchi,NG,10.50,10.00,1595012017,74.21,82,87,2.01
542,542,Havelock,US,34.88,-76.90,1595011828,89.01,69,12,1.01
543,543,Sioux Falls,US,43.55,-96.70,1595011907,87.80,66,40,14.99
544,544,Naze,JP,28.37,129.48,1595012017,80.60,88,75,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
gmaps.configure(api_key=g_key)


locations = cities[["Latitude", "Longitude"]]
humidity = cities["Humidity (%)"]
fig = gmaps.figure(zoom_level=1.6, center= (20, 0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1,
                                )

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_city = cities.loc[(cities["Temperature"]<80) & (cities["Temperature"]>75) & (cities["Wind Speed (mph)"]<10)& (cities["Cloudiness (%)"]==0) & (cities["Wind Speed (mph)"]<9)].dropna()
ideal_city

,Unnamed: 0,City,Country,Latitude,Longitude,Date Time,Temperature,Humidity (%),Cloudiness (%),Wind Speed (mph)
138,138,Erzin,TR,36.96,36.20,1595011969,79.00,78,0,2.48
165,165,Sirte,LY,31.21,16.59,1595011972,78.01,81,0,8.81
317,317,Botwood,CA,49.15,-55.35,1595011990,75.99,36,0,1.01
410,410,San Miguel de Tucumán,AR,-26.82,-65.22,1595012001,75.20,38,0,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_city[['City','Country','Latitude','Longitude']].copy()
#renaming columns so provided code works
hotel_df.rename(columns = {"Latitude":"Lat", "Longitude": "Lng"}, inplace=True)
hotel_df["Hotel Name"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
138,Erzin,TR,36.96,36.20,
165,Sirte,LY,31.21,16.59,
317,Botwood,CA,49.15,-55.35,
410,San Miguel de Tucumán,AR,-26.82,-65.22,


In [11]:
# geocoordinates
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng =row['Lng']
    params['location'] = f"{lat},{lng}"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
    except (KeyError, IndexError):
        print(f"no hotel in {index}")
print(hotel_df)
hotel_df.to_csv("output_data/hotel_data")

                      City Country    Lat    Lng            Hotel Name
138                  Erzin      TR  36.96  36.20          Artemis Otel
165                  Sirte      LY  31.21  16.59            City Hotel
317                Botwood      CA  49.15 -55.35       Exploits Hostel
410  San Miguel de Tucumán      AR -26.82 -65.22  Dallas Hotel Tucuman


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))